<a href="https://colab.research.google.com/github/eric15342335/realfill/blob/main/train_realfill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/eric15342335/realfill
%cd realfill

Cloning into 'realfill'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 288 (delta 49), reused 32 (delta 25), pack-reused 216 (from 1)
Receiving objects: 100% (288/288), 1.26 MiB | 8.60 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/realfill


In [2]:
!curl -L https://github.com/eric15342335/realfill/releases/download/dataset/realfill_data_release_full.zip -o realfill_data_release_full.zip
!unzip -q realfill_data_release_full.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  452M  100  452M    0     0  8659k      0  0:00:53  0:00:53 --:--:-- 8182k


In [3]:
%pwd
%ls -lh

total 454M
drwxr-xr-x 4 root root 4.0K Apr 19 16:20 data/
-rw-r--r-- 1 root root 2.5K Apr 19 16:20 infer.py
-rw-r--r-- 1 root root 1.1K Apr 19 16:20 LICENSE
drwxr-xr-x 3 root root 4.0K Apr 19 16:21 __MACOSX/
-rw-r--r-- 1 root root  383 Apr 19 16:20 README.md
-rw-r--r-- 1 root root 5.9K Apr 19 16:20 README-Realfill.md
drwxr-xr-x 4 root root 4.0K May 30  2024 realfill_data_release_full/
-rw-r--r-- 1 root root 453M Apr 19 16:21 realfill_data_release_full.zip
-rw-r--r-- 1 root root   97 Apr 19 16:20 requirements.txt
-rw-r--r-- 1 root root 6.5K Apr 19 16:20 train_realfill.ipynb
-rw-r--r-- 1 root root  37K Apr 19 16:20 train_realfill.py


In [4]:
%pip install -U -r requirements.txt --progress-bar off

  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.5.82
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.6.82
    Uninstalling nvidia-curand-cu12-10.3.6.82:
      Successfully uninstalled nvidia-curand-cu12-10.3.6.82
  Attempting uninstall: nvidia-cufft-cu12
    Found existing installation: nvidia-cufft-cu12 11.2.3.61
    Uninstalling nvidia-cufft-cu12-11.2.3.61:
      Successfully uninstalled nvidia-cufft-cu12-11.2.3.61
  Attempting uninstall: nvidia-cuda-runtime-cu12
    Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
    Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
  Attempting uninstall: nvidia-cuda-nvrtc-cu12
    Found existing installation: nvidia-cuda-nvrtc-cu12 12.5.82
    Uninstalling nvid

In [5]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [10]:
import os

# Toggle switch - set to True for Google Drive storage, False for local storage
USE_DRIVE_STORAGE = True

# Base directory for Google Drive storage
DRIVE_BASE_DIR = "/content/drive/MyDrive/RealFill"

# Keep your original environment variable setup
os.environ["DATASET"] = "realfill_data_release_full"
os.environ["MODEL_NAME"] = "stabilityai/stable-diffusion-2-inpainting"
os.environ["BENCHMARK"] = "RealBench"  # Replace with "Qualitative" if needed
os.environ["DATASET_NUMBER"] = "23"

# Construct paths based on the storage toggle
if USE_DRIVE_STORAGE:
    # Use Google Drive for storage
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ["OUTPUT_DIR"] = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-model'
    os.environ["OUTPUT_IMG_DIR"] = f'{DRIVE_BASE_DIR}/{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'
else:
    # Use local Colab storage (original paths)
    os.environ["OUTPUT_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-model'
    os.environ["OUTPUT_IMG_DIR"] = f'{os.environ["BENCHMARK"]}-{os.environ["DATASET_NUMBER"]}-results'

# The remaining variable paths stay the same
os.environ["TRAIN_DIR"] = f'{os.environ["DATASET"]}/{os.environ["BENCHMARK"]}/{os.environ["DATASET_NUMBER"]}'
os.environ["VAL_IMG"] = f'{os.environ["TRAIN_DIR"]}/target/target.png'
os.environ["VAL_MASK"] = f'{os.environ["TRAIN_DIR"]}/target/mask.png'

# Create the necessary directories
os.makedirs(os.environ["OUTPUT_DIR"], exist_ok=True)
os.makedirs(os.environ["OUTPUT_IMG_DIR"], exist_ok=True)

print(f"Using {'Google Drive' if USE_DRIVE_STORAGE else 'local Colab'} storage")
print(f"Output directory: {os.environ['OUTPUT_DIR']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Google Drive storage
Output directory: /content/drive/MyDrive/RealFill/RealBench-23-model


In [ ]:
!accelerate launch train_realfill.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --train_data_dir=$TRAIN_DIR \
  --output_dir=$OUTPUT_DIR \
  --resolution=512 \
  --train_batch_size=16 \
  --gradient_accumulation_steps=1 \
  --unet_learning_rate=2e-4 \
  --text_encoder_learning_rate=4e-5 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=100 \
  --max_train_steps=2000 \
  --lora_rank=8 \
  --lora_dropout=0.1 \
  --lora_alpha=16 \
  --resume_from_checkpoint="latest" \
  --report_to tensorboard \
  --validation_steps 100 \
  --checkpointing_steps 100 \
  --mixed_precision="bf16"

2025-04-19 16:27:24.921530: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745080044.940835    2696 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745080044.946721    2696 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 16:27:24.968449: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
04/19/2025 16:27:28 - INFO - __main__ - Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0

In [ ]:
!accelerate launch infer.py \
    --model_path=$OUTPUT_DIR \
    --validation_image=$VAL_IMG \
    --validation_mask=$VAL_MASK \
    --output_dir=$OUTPUT_IMG_DIR

In [ ]:
# Zip final inference results
!zip -r $OUTPUT_IMG_DIR.zip $OUTPUT_IMG_DIR
# Zip tensorboard logs
!zip -r $OUTPUT_DIR-tensorboard.zip $OUTPUT_DIR/logs